In [0]:
oct_df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)
oct_df.show(5)

In [0]:
%sql
SHOW VOLUMES IN workspace.ecommerce;

In [0]:
%sql
CREATE VOLUME workspace.ecommerce.delta;

In [0]:
# Create Delta tables (PySpark)

oct_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/ecommerce/delta/ecommerce_events")


In [0]:
# Create managed table
oct_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.ecommerce.ecommerce_events_oct")


In [0]:
# SQL approach
spark.sql(""" CREATE TABLE workspace.ecommerce.ecommerce_events_delta
USING DELTA
AS
SELECT * FROM workspace.ecommerce.ecommerce_events_oct;
 """)

In [0]:
#schema enforcement

wrong_schema_df = spark.createDataFrame(
    [("a", "b", "c")],
    ["x", "y", "z"]
)


try:
    wrong_schema_df.write \
        .format("delta") \
        .mode("append") \
        .save("/delta/ecommerce_events")
except Exception as e:
    print("Schema enforcement works!")
    print(e)

Handle duplicate inserts

In [0]:
nov_df = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",header=True,inferSchema=True)

nov_df.write.format("delta").mode("append").save("/Volumes/workspace/ecommerce/delta/ecommerce_events")


In [0]:
df = spark.read.format("delta").load("/Volumes/workspace/ecommerce/delta/ecommerce_events")

print("total count: ",df.count())            # total rows
print("unique rows: ",df.distinct().count()) # unique rows


In [0]:
dedup_df = df.dropDuplicates()

In [0]:
dedup_df.write.format("delta").mode("overwrite").save("/Volumes/workspace/ecommerce/delta/ecommerce_events")

In [0]:
df_clean = spark.read.format("delta").load("/Volumes/workspace/ecommerce/delta/ecommerce_events")

print("total count:", df_clean.count())
print("unique rows:", df_clean.distinct().count())
